# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case=9_bert_training", "case/server=malicious-transformer",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case bert_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [20]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [4] # This is the sequence length

cfg.case.model = "bert-sanity-check" #  "huawei-noah/TinyBERT_General_4L_312D"
cfg.case.data.tokenizer = "bert-base-uncased"
cfg.case.data.vocab_size = 30522
cfg.case.data.disable_mlm=False


cfg.case.server.provide_public_buffers = True
cfg.case.server.has_external_data = True
cfg.case.server.pretrained = False


cfg.case.server.param_modification.v_length = 32
cfg.case.server.param_modification.eps = 1e-8
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 0
cfg.case.server.param_modification.sequence_token_weight = 1
cfg.case.server.param_modification.measurement_scale = 1

cfg.case.server.param_modification.equalize_token_weight = 10
cfg.case.server.param_modification.reset_embedding = True

cfg.attack.token_strategy="embedding-norm" # "decoder-bias" is only nice for bert if disable_mlm=True

cfg.attack.embedding_token_weight = 0.25

cfg.attack.impl.dtype="float"

### Instantiate all parties

In [21]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture bert-sanity-check loaded with 109,514,298 parameters and 1,024 buffers.
Overall this is a data ratio of 27378574:1 for target shape [1, 4] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 1

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: bert-sanity-check
        model state: default
        public bu

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [22]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Found attention of shape torch.Size([768, 768]).
Found attention of shape torch.Size([768, 768]).
Computing feature distribution before the probe layer Linear(in_features=768, out_features=3072, bias=True) from external data.
Feature mean is 0.09274651855230331, feature std is 0.9499862194061279.
Computing user update in model mode: eval.


In [23]:
user.print(true_user_data)

[CLS] the tower building


In [24]:
cfg.attack.impl.dtype="float"

# Reconstruct user data

In [25]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

IndexError: argmax(): Expected reduction dim to be specified for input.numel() == 0.

In [ ]:
metrics

In [10]:
torch.nn.functional.gelu(torch.as_tensor([0.5, 0.1, 0.01, -0.01, -1, -5]))

tensor([ 3.4573e-01,  5.3983e-02,  5.0399e-03, -4.9601e-03, -1.5866e-01,
        -1.4901e-06])